In [1]:
import json
import ast
import re

import pandas as pd
import xarray as xr

In [2]:
df = pd.read_csv("_groene_mient_results_0.csv", index_col=1).drop(["Unnamed: 0"], axis=1)

In [3]:
df.loc["2021-01-01", :]

,M1: realised_demand,M2: scheduled_demand,M3: shifted_load,M4: total_generation,M5: savings_on_ToD,M6: energy_costs
date,,,,,,
2021-01-01,"{'hh3_consumption [kWh]_1': 35.63198185276816,...","{'hh3_consumption [kWh]_1': 37.19186982076816,...","{""hh3_consumption [kWh]_1"": 0, ""hh1_consumptio...","{""<class 'model.agents.Solar'>"": 59.2757427839...","{""hh3_consumption [kWh]_1"": 0.0, ""hh1_consumpt...","{""hh3_consumption [kWh]_1"": 11.225956249240364..."
2021-01-01,"{'hh3_consumption [kWh]_1': 35.63198185276816,...","{'hh3_consumption [kWh]_1': 37.19186982076816,...","{""hh3_consumption [kWh]_1"": 0, ""hh1_consumptio...","{""<class 'model.agents.Solar'>"": 59.2757427839...","{""hh3_consumption [kWh]_1"": 0.0, ""hh1_consumpt...","{""hh3_consumption [kWh]_1"": 11.225956249240364..."
2021-01-01,"{'hh3_consumption [kWh]_1': 35.63198185276816,...","{'hh3_consumption [kWh]_1': 37.19186982076816,...","{""hh3_consumption [kWh]_1"": 0, ""hh1_consumptio...","{""<class 'model.agents.Solar'>"": 59.2757427839...","{""hh3_consumption [kWh]_1"": 0.0, ""hh1_consumpt...","{""hh3_consumption [kWh]_1"": 11.225956249240364..."
2021-01-01,"{'hh3_consumption [kWh]_1': 35.63198185276816,...","{'hh3_consumption [kWh]_1': 37.19186982076816,...","{""hh3_consumption [kWh]_1"": 0, ""hh1_consumptio...","{""<class 'model.agents.Solar'>"": 59.2757427839...","{""hh3_consumption [kWh]_1"": 0.0, ""hh1_consumpt...","{""hh3_consumption [kWh]_1"": 11.225956249240364..."
2021-01-01,"{'hh3_consumption [kWh]_1': 35.63198185276816,...","{'hh3_consumption [kWh]_1': 37.19186982076816,...","{""hh3_consumption [kWh]_1"": 0, ""hh1_consumptio...","{""<class 'model.agents.Solar'>"": 59.2757427839...","{""hh3_consumption [kWh]_1"": 0.0, ""hh1_consumpt...","{""hh3_consumption [kWh]_1"": 11.225956249240364..."
2021-01-01,"{'hh3_consumption [kWh]_1': 35.63198185276816,...","{'hh3_consumption [kWh]_1': 37.19186982076816,...","{""hh3_consumption [kWh]_1"": 0, ""hh1_consumptio...","{""<class 'model.agents.Solar'>"": 59.2757427839...","{""hh3_consumption [kWh]_1"": 0.0, ""hh1_consumpt...","{""hh3_consumption [kWh]_1"": 11.225956249240364..."
2021-01-01,"{'hh3_consumption [kWh]_1': 35.63198185276816,...","{'hh3_consumption [kWh]_1': 37.19186982076816,...","{""hh3_consumption [kWh]_1"": 0, ""hh1_consumptio...","{""<class 'model.agents.Solar'>"": 59.2757427839...","{""hh3_consumption [kWh]_1"": 0.0, ""hh1_consumpt...","{""hh3_consumption [kWh]_1"": 11.225956249240364..."
2021-01-01,"{'hh3_consumption [kWh]_1': 35.63198185276816,...","{'hh3_consumption [kWh]_1': 37.19186982076816,...","{""hh3_consumption [kWh]_1"": 0, ""hh1_consumptio...","{""<class 'model.agents.Solar'>"": 59.2757427839...","{""hh3_consumption [kWh]_1"": 0.0, ""hh1_consumpt...","{""hh3_consumption [kWh]_1"": 11.225956249240364..."
2021-01-01,"{'hh3_consumption [kWh]_1': 35.63198185276816,...","{'hh3_consumption [kWh]_1': 37.19186982076816,...","{""hh3_consumption [kWh]_1"": 0, ""hh1_consumptio...","{""<class 'model.agents.Solar'>"": 59.2757427839...","{""hh3_consumption [kWh]_1"": 0.0, ""hh1_consumpt...","{""hh3_consumption [kWh]_1"": 11.225956249240364..."


In [4]:
len(df)

3650

In [5]:
df = df[~df.index.duplicated()]

In [6]:
len(df)

364

In [7]:
def parse_json(i):
    try:
        json.loads(i)
    except ValueError:
        i = ast.literal_eval(re.search('({.+})', i).group(0))
    else:
        i = json.loads(i)
    return i

In [8]:
ds = xr.Dataset({col: pd.DataFrame(df[col].apply(parse_json)).to_xarray().to_array() for col in df.columns if ":" in col})

In [9]:
ds  # xarray.Dataset

<xarray.Dataset>
Dimensions:               (variable: 6, date: 364)
Coordinates:
  * variable              (variable) <U20 'M1: realised_demand' ... 'M6: ener...
  * date                  (date) object '2021-01-01' ... '2021-12-30'
Data variables:
    M1: realised_demand   (variable, date) object {'hh3_consumption [kWh]_1':...
    M2: scheduled_demand  (variable, date) object nan nan nan ... nan nan nan
    M3: shifted_load      (variable, date) object nan nan nan ... nan nan nan
    M4: total_generation  (variable, date) object nan nan nan ... nan nan nan
    M5: savings_on_ToD    (variable, date) object nan nan nan ... nan nan nan
    M6: energy_costs      (variable, date) object nan ... {'hh3_consumption [...

In [10]:
ds.to_dataframe()  # turn the dataset back into a multi-indexed pandas.DataFrame

M1: realised_demand  \
                    date                                                            
M1: realised_demand 2021-01-01  {'hh3_consumption [kWh]_1': 35.63198185276816,...   
                    2021-01-02  {'hh3_consumption [kWh]_1': 34.26672802920719,...   
                    2021-01-03  {'hh3_consumption [kWh]_1': 35.225524317207174...   
                    2021-01-04  {'hh3_consumption [kWh]_1': 35.287237117207184...   
                    2021-01-05  {'hh3_consumption [kWh]_1': 33.9540933391631, ...   
...                                                                           ...   
M6: energy_costs    2021-12-26                                                NaN   
                    2021-12-27                                                NaN   
                    2021-12-28                                                NaN   
                    2021-12-29                                                NaN   
                    2021-12-30                                                NaN   

                               M2: scheduled_demand M3: shifted_load  \
                    date                                               
M1: realised_demand 2021-01-01                  NaN              NaN   
                    2021-01-02                  NaN              NaN   
                    2021-01-03                  NaN              NaN   
                    2021-01-04                  NaN              NaN   
                    2021-01-05                  NaN              NaN   
...                                             ...              ...   
M6: energy_costs    2021-12-26                  NaN              NaN   
                    2021-12-27                  NaN              NaN   
                    2021-12-28                  NaN              NaN   
                    2021-12-29                  NaN              NaN   
                    2021-12-30                  NaN              NaN   

                               M4: total_generation M5: savings_on_ToD  \
                    date                                                 
M1: realised_demand 2021-01-01                  NaN                NaN   
                    2021-01-02                  NaN                NaN   
                    2021-01-03                  NaN                NaN   
                    2021-01-04                  NaN                NaN   
                    2021-01-05                  NaN                NaN   
...                                             ...                ...   
M6: energy_costs    2021-12-26                  NaN                NaN   
                    2021-12-27                  NaN                NaN   
                    2021-12-28                  NaN                NaN   
                    2021-12-29                  NaN                NaN   
                    2021-12-30                  NaN                NaN   

                                                                 M6: energy_costs  
                    date                                                           
M1: realised_demand 2021-01-01                                                NaN  
                    2021-01-02                                                NaN  
                    2021-01-03                                                NaN  
                    2021-01-04                                                NaN  
                    2021-01-05                                                NaN  
...                                                                           ...  
M6: energy_costs    2021-12-26  {'hh3_consumption [kWh]_1': 18.149573344893035...  
                    2021-12-27  {'hh3_consumption [kWh]_1': 19.19839864724843,...  
                    2021-12-28  {'hh3_consumption [kWh]_1': 18.575140176517227...  
                    2021-12-29  {'hh3_consumption [kWh]_1': 18.172060597658476...  
                    2021-12-30  {'hh3_consumptio